In [6]:
# for cpu table (i.e. ProcessingUnits)
# using dict
# FOR LOCAL HOST
import platform, subprocess, re, pymysql, pandas as pd
class ComputerInfo(object):
    
    def __init__(self,computer_name):
        self.computer_name = computer_name
        
    def get_thread_count(self):
        if platform.system()=="Linux":
            command = "grep siblings /proc/cpuinfo"
            new_info = subprocess.check_output(command,shell=True).strip().decode()
            split = new_info.split("\n")
            words_in_line = split[0].split()
            return words_in_line[0], words_in_line[len(words_in_line)-1]        
        
    def get_processor_name(self):
        processor_dict = {} # cpu(S) is number of threads per socket 
        names_list = ['siblings','Socket(s)','Model name','Core(s) per socket','CPU MHz', 'CPU max MHz']
        output_list = []
        if platform.system() == "Linux":
            command = "lscpu"
            all_info = subprocess.check_output(command, shell=True).strip().decode()
            split_by_line = all_info.split("\n") # split into new string when on new line
            for line in range(0,len(split_by_line)): # line is line number
                string = split_by_line[line]
                no_spaces = re.sub("\s+"," ",string,flags=re.UNICODE)
                list_in_line = re.split(":",no_spaces) # gets rid of duplicate spaces and joins words
                processor_dict.update({list_in_line[0]:list_in_line[1]})
            out,threads = self.get_thread_count()
            processor_dict.update({out:threads})
            for name in names_list:
                   output_list.append(processor_dict.setdefault(name,None))

            data = list(zip(names_list,output_list))
            df = pd.DataFrame(data,columns=['Item','Output'])
            return df
        
    def info_to_mysql(self): # get to work for actual tables now 8/1/19
        processor_table = self.get_processor_name()
        connection = pymysql.connect(host='localhost',
                                 port=int(3306),
                                 user='root',
                                 db ='Find Computer',
                                 passwd = 'password',
                                 charset='utf8mb4') # did I default charset in database to utf8? 
        processor_record = "INSERT INTO Test (A,B,C,D,E,j) VALUES (%s,%s,%s,%s,%s,%s)"
                                               
        cursor = connection.cursor()
        cursor.execute(processor_record,(processor_table.loc[0,'Output'],
                                         processor_table.loc[1,'Output'],
                                         processor_table.loc[2,'Output'],
                                         processor_table.loc[3,'Output'],
                                         processor_table.loc[4,'Output'],
                                         processor_table.loc[5,'Output'])) 
        connection.commit()
        connection.close()
a = ComputerInfo("cannon")
a.get_processor_name()
#a.info_to_mysql()

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                12
On-line CPU(s) list:   0-11
Thread(s) per core:    2
Core(s) per socket:    6
Socket(s):             1
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 44
Model name:            Intel(R) Xeon(R) CPU           X5680  @ 3.33GHz
Stepping:              2
CPU MHz:               1600.000
CPU max MHz:           3334.0000
CPU min MHz:           1600.0000
BogoMIPS:              6649.55
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              12288K
NUMA node0 CPU(s):     0-11


,Item,Output
0,siblings,12
1,Socket(s),1
2,Model name,Intel(R) Xeon(R) CPU X5680 @ 3.33GHz
3,Core(s) per socket,6
4,CPU MHz,1600.000
5,CPU max MHz,3334.0000


In [1]:
# for cpu table (i.e. ProcessingUnits)
# using dict
# work for multiple computers 
# NEED TO FIX 
import platform, subprocess, re, pymysql, pandas as pd,paramiko
class ComputerInfo(object):
    
    def __init__(self,computer_name):
        self.computer_name = computer_name
        
    def ssh_to_comp(self,computer_name):
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect(self.computer_name, username='wmcgrath') # enter password
        ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('lscpu')
        return ssh_stdout.read().splitlines()
    
    def get_thread_count(self):
        if platform.system()=="Linux":
            command = "grep siblings /proc/cpuinfo"
            new_info = subprocess.check_output(command,shell=True).strip().decode()
            split = new_info.split("\n")
            words_in_line = split[0].split()
            return words_in_line[0], words_in_line[len(words_in_line)-1]        
        
    def get_processor_name(self):
        processor_dict = {} # cpu(S) is number of threads per socket 
        names_list = ['siblings','Socket(s)','Model name','Core(s) per socket','CPU MHz', 'CPU max MHz']
        output_list = []
        #ssh_info = self.ssh_to_comp(self.computer_name)
        if platform.system() == "Linux":
            for line in range(0,len(ssh_info)): # line is line number
                string = str(ssh_info[line])
                no_spaces = re.sub("\s+"," ",string,flags=re.UNICODE)
                list_in_line = re.split(":",no_spaces) # gets rid of duplicate spaces and joins words
                processor_dict.update({list_in_line[0]:list_in_line[1]})
            out,threads = self.get_thread_count()
            processor_dict.update({out:threads})
            for name in names_list:
                    output_list.append(processor_dict.setdefault(name,None))

            data = list(zip(names_list,output_list))
            df = pd.DataFrame(data,columns=['Item','Output'])
            return df
        
    def info_to_mysql(self): # get to work for actual tables now 8/1/19
        processor_table = self.get_processor_name()
        connection = pymysql.connect(host='localhost',
                                 port=int(3306),
                                 user='root',
                                 db ='Find Computer',
                                 passwd = 'password',
                                 charset='utf8mb4') # did I default charset in database to utf8? 
        processor_record = "INSERT INTO Test (A,B,C,D,E,j) VALUES (%s,%s,%s,%s,%s,%s)"
                                               
        cursor = connection.cursor()
        cursor.execute(processor_record,(processor_table.loc[0,'Output'],
                                         processor_table.loc[1,'Output'],
                                         processor_table.loc[2,'Output'],
                                         processor_table.loc[3,'Output'],
                                         processor_table.loc[4,'Output'],
                                         processor_table.loc[5,'Output'])) 
        connection.commit()
        connection.close()
a = ComputerInfo('bertha')
a.ssh_to_comp('bertha')
#a.get_processor_name()

[b'Architecture:          x86_64',
 b'CPU op-mode(s):        32-bit, 64-bit',
 b'Byte Order:            Little Endian',
 b'CPU(s):                32',
 b'On-line CPU(s) list:   0-31',
 b'Thread(s) per core:    2',
 b'Core(s) per socket:    8',
 b'Socket(s):             2',
 b'NUMA node(s):          2',
 b'Vendor ID:             GenuineIntel',
 b'CPU family:            6',
 b'Model:                 45',
 b'Model name:            Intel(R) Xeon(R) CPU E5-2687W 0 @ 3.10GHz',
 b'Stepping:              7',
 b'CPU MHz:               3399.949',
 b'CPU max MHz:           3800.0000',
 b'CPU min MHz:           1200.0000',
 b'BogoMIPS:              6186.66',
 b'Virtualization:        VT-x',
 b'L1d cache:             32K',
 b'L1i cache:             32K',
 b'L2 cache:              256K',
 b'L3 cache:              20480K',
 b'NUMA node0 CPU(s):     0-7,16-23',
 b'NUMA node1 CPU(s):     8-15,24-31']

In [10]:
gg = b'333'
print(gg.decode('utf-8'))

333


In [3]:
# for cpu table (i.e. ProcessingUnits)
# using dict
# work for multiple computers (including cannon)
# WORKS (updated version) 8/13/19 
import platform, subprocess, re, pymysql, pandas as pd,paramiko
class ComputerInfo(object):
    
    def __init__(self,computer_name):
        self.computer_name = computer_name
        
    def ssh_to_comp(self,computer_name): # do i need a computer name and should i make the commands inputs so 
        ssh = paramiko.SSHClient()       # i use this method for multiple commands 
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect(self.computer_name, username='wmcgrath') 
        ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('lscpu')        
        other_stdin, other_stdout, other_stderr = ssh.exec_command('grep siblings /proc/cpuinfo')     
        return ssh_stdout.read().splitlines(), other_stdout.readline().splitlines()
          
    def get_cpu_info(self): #,computer_name):
        processor_dict = {} # cpu(S) is number of threads per socket 
        names_list = ['siblings','Socket(s)','Model name','Core(s) per socket','CPU min MHz', 'CPU max MHz']
        output_list = []
        ssh_info, other_info = self.ssh_to_comp(self.computer_name)           
        if platform.system() == "Linux":
            for indx in ssh_info: 
                str_form = indx.decode('utf-8')
                no_spaces = re.sub("\s+"," ",str_form,flags=re.UNICODE)
                list_in_line = re.split(":",no_spaces) # gets rid of duplicate spaces and joins words
                processor_dict.update({list_in_line[0]:list_in_line[1]})
            for other_indx in other_info:
                other_form = re.sub("\s+","",other_indx,flags=re.UNICODE)
                other_in_line = re.split(":",other_form)
                processor_dict.update({other_in_line[0]:other_in_line[1]})
                
            for name in names_list:
                    output_list.append(processor_dict.setdefault(name,None)) # looks for key's value and appends to list
            data = list(zip(names_list,output_list)) # makes new list to feed into dataframe
            df = pd.DataFrame(data,columns=['Item','Output'])
            return df
        
    def info_to_mysql(self): # get to work for actual tables now 8/1/19
        cpu_table = self.get_cpu_info()
        connection = pymysql.connect(host='localhost',
                                 port=int(3306),
                                 user='root',
                                 db ='Find Computer',
                                 passwd = 'password',
                                 charset='utf8mb4') # did I default charset in database to utf8? 
        cpu_record = "INSERT INTO Test (A,B,C,D,E,j) VALUES (%s,%s,%s,%s,%s,%s)"
                                               
        cursor = connection.cursor()
        cursor.execute(cpu_record,(cpu_table.loc[0,'Output'],
                                         cpu_table.loc[1,'Output'],
                                         cpu_table.loc[2,'Output'],
                                         cpu_table.loc[3,'Output'],
                                         cpu_table.loc[4,'Output'],
                                         cpu_table.loc[5,'Output'])) 
        connection.commit()
        connection.close()
a = ComputerInfo('bertha')
a.get_cpu_info()

,Item,Output
0,siblings,16
1,Socket(s),2
2,Model name,Intel(R) Xeon(R) CPU E5-2687W 0 @ 3.10GHz
3,Core(s) per socket,8
4,CPU min MHz,1200.0000
5,CPU max MHz,3800.0000


In [2]:
a.info_to_mysql()

In [52]:
# using dict
# work for multiple computers (including cannon)
# WORKS (updated version) 9/6/19
# get line numbers 'ctrl m' then 'l'
import platform, subprocess, re, pymysql, pandas as pd,paramiko #, json ? for parsing lists 
class ComputerInfo(object):
    
    def __init__(self,computer_name):
        self.computer_name = computer_name
        
    def ssh_to_comp(self,computer_name,command): # do i need a computer name and should i make the commands inputs so 
        ssh = paramiko.SSHClient()       # i use this method for multiple commands 
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect(self.computer_name, username='wmcgrath') 
        ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(command)         
        return ssh_stdout.read().splitlines() 
          
    def get_cpu_info(self): #,computer_name):
        processor_dict = {} # cpu(S) is number of threads per socket 
        names_list = ['siblings','Socket(s)','Model name','Core(s) per socket','CPU min MHz', 'CPU max MHz']
        output_list = []
        command_one = 'lscpu'
        command_two = 'grep siblings /proc/cpuinfo'
        ssh_info = self.ssh_to_comp(self.computer_name,command_one)       
        other_info = self.ssh_to_comp(self.computer_name,command_two)     
        if platform.system() == "Linux":
            for indx in ssh_info: 
                str_form = indx.decode('utf-8') # can only decode one line at a time so can't decode a list
                no_spaces = re.sub("\s+"," ",str_form,flags=re.UNICODE)
                list_in_line = re.split(":",no_spaces) # gets rid of duplicate spaces and joins words
                processor_dict.update({list_in_line[0]:list_in_line[1]})
            for other_indx in other_info:
                decode_form = other_indx.decode('utf-8')
                other_form = re.sub("\s+","",decode_form,flags=re.UNICODE)
                other_in_line = re.split(":",other_form)
                processor_dict.update({other_in_line[0]:other_in_line[1]})
                
            for name in names_list:
                    output_list.append(processor_dict.setdefault(name,None)) # looks for key's value and appends to list
            data = list(zip(names_list,output_list)) # makes new list to feed into dataframe
            df = pd.DataFrame(data,columns=['Item','Output'])
            return df
        
    def get_gpu_info(self): # WHERE?!?! if all info is parsed the same then only need one of these methods 
        gpu_dict = {}
        names_list = [...] #fix
        output_list = [] 
        command_one = 'lspci | grep VGA | cut -d ":" -f3'
        gpu_info = self.ssh_to_comp(self.computer_name, command_one)
        if platform.system() == "Linux":
            for indx in gpu_info:
                str_form = indx.decode('utf-8') # gets rid of b char 
                split_form = re.search('\[(.*?)\]',str_form, flags=re.UNICODE) # finds that string 
                each_word = split_form.group(0) # group(0) stands for all matched string
                no_brackets = re.sub("\[|\]","",each_word,flags=re.UNICODE)
                output_list.extend([no_brackets])
            names = ['GPU Model']
            zip_list = list(zip(names,output_list))
            df = pd.DataFrame(zip_list,columns=['Item','Output'])
            if len(output_list) > 1:  
                df.at[0,'Output'] = output_list
            return df # should i print mult gpu names in one cell and should they be in a list or in a col 
                      # where each name is on a new line in that one cell ?!?!?
            
    def get_serial_number(self): # and get lab comp is in       
        # use qhost command to get list of comp names*****
        cnl_comp = []
        command_one = 'qhost >> hostnames.txt'
        command_two = 'qhost > hostnames.txt'
        command_three = 'cat hostnames.txt'
        command_list = self.ssh_to_comp(self.computer_name,command_three)
        
        for indx in command_list:
            str_form = indx.decode('utf-8')
            no_spaces = re.sub("\s+"," ",str_form,flags=re.UNICODE)
            split_lines = re.split("\s",no_spaces)
            cnl_comp.append(split_lines[0])
        cnl_comp = cnl_comp[2::]
            
        if self.computer_name in cnl_comp: 
            lab_name = 'CNL'
            #add all SHOULD THEY BE NUMBERS OR TEXT?    
            serial_numbers = {'cajal':'130870-4','hooke':'NEED', 'holmes':'126704-1','pitts':'126606-1',
                              'kepler':'133639-1','lorenz':'133639-2','cannon':'126704-2','katz':'130870-2',
                              'Julia':'154043', 'Eccles':'1232323','Ullrich':'131428-2', 'Stumpf':'136207',
                              'pringles': 'NEED', 'wise':'NEED','stokes': '126606-2', 'hopper':'130870-3',
                              'roche': '131428-1', 'helmholtz': 'NEED', 'scarlett':'1511623-3', 'wells':'151623-4',
                              'morton':'151623-2', 'leroy':'151623-1','dale':'136484', 'hitzig':'144461',
                              'schleck':'126606-3','hartley':'125908-1', 'falloppio': '125288'} 
            
            # get Salk ID
            Salk_ID = {}
            return serial_numbers[self.computer_name], lab_name
            
    def get_main_table_info(self):
        main_dict = {}
        output_list = [] 
        serial_number, lab_name = self.get_serial_number()
        command_one = 'cat /etc/os-release'
        os_info = self.ssh_to_comp(self.computer_name,command_one)        
        if platform.system() == "Linux":
            for indx in os_info:
                str_form = indx.decode('utf-8')
                str_split = re.split('=',str_form)
                if str_split[0] == 'PRETTY_NAME':
                    os_version = str_split[1].replace('\"','') # gets rid of quotes on string
                if str_split[0] == 'VERSION':
                    version = str_split[1].replace('\"','')
                    break                 
            main_dict.update({'Name':self.computer_name, 'Serial Number':serial_number, 'Lab':lab_name, 
                             'Operating System':os_version, 'OS Version':version})
            df = pd.DataFrame(list(main_dict.items()),columns=['Item','Output'])
            return df
                                  
    def info_to_mysql(self): # got to work for actual tables now 8/1/19
        cpu_table = self.get_cpu_info()
        connection = pymysql.connect(host='localhost',
                                 port=int(3306),  #doesnt need int lol 
                                 user='root',
                                 db ='Find Computer',
                                 passwd = 'RedTailedShark',
                                 charset='utf8mb4') # did I default charset in database to utf8? 
        cpu_record = "INSERT INTO Test (A,B,C,D,E,j) VALUES (%s,%s,%s,%s,%s,%s)"
                                               
        cursor = connection.cursor()
        cursor.execute(cpu_record,(cpu_table.loc[0,'Output'],
                                         cpu_table.loc[1,'Output'],
                                         cpu_table.loc[2,'Output'],
                                         cpu_table.loc[3,'Output'],
                                         cpu_table.loc[4,'Output'],
                                         cpu_table.loc[5,'Output'])) 
        connection.commit()
        connection.close()
a = ComputerInfo('cannon')
#a.get_cpu_info()
a.get_serial_number()

('126704-2', 'CNL')

In [ ]:
a.info_to_mysql()